In [ ]:
!pip3 install http://download.pytorch.org/whl/cu92/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())



0.4.1
True


In [ ]:
!pip3 install fastprogress

In [ ]:
path = 'train_images.npy'
labels = 'train_labels.csv'
test_path = 'test_images.npy'

In [ ]:
import torch
torch.cuda.set_device(0)

In [ ]:
import numpy as np
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

In [ ]:
data = np.load(path, encoding='bytes')
test_data = np.load(test_path, encoding='bytes')

In [ ]:
### normalization
def stats(imgs): 
  means = []
  stds = []
  for i in range(len(imgs)):
    means.append(np.mean(imgs[i]))

  for i in range(len(imgs)):
    stds.append(np.std(imgs[i]))

  mean = sum(means)/len(means)
  std = sum(stds)/len(stds)

  for i in range(len(imgs)):
    imgs[i] = (imgs[i] - mean)/std
  
  return imgs

In [ ]:
### split the data
np.random.shuffle(data)
split = 0.25
length = len(data)
test_idx, train_idx = data[:int(length*split)], data[int(length*split):] 

In [ ]:
### normalization
# data[:int(length*split), 1] = stats(data[:int(length*split), 1])
# data[int(length*split):, 1] = stats(data[int(length*split):, 1])

In [ ]:
class MyDataset(Dataset):

    def __init__(self, csv_file, data, transform=None, is_test=False):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.lbs = pd.read_csv(csv_file)
        self.labels = self.lbs['Category'].values.tolist()
        self.classes = sorted(set(self.lbs['Category'].values.tolist()))
        self.class2idx = {cl:idx for idx, cl in enumerate(self.classes)}
        self.data = data
        self.transform = transform
        self.is_test=is_test

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx][1].astype(np.float32)
        img = img.reshape(100, 100)
        img = np.stack([img]*3, axis=2)
        img = Image.fromarray(img.astype('uint8'), 'RGB')
        
        if self.transform:
            img = self.transform(img)
            
        if self.is_test:
            return img

        y = self.labels[self.data[idx][0]]
        y = self.class2idx[y]


        return img, y

In [ ]:
### image Agmentation
transform_train = transforms.Compose([
                                 transforms.Resize(224),
                                 transforms.RandomRotation(10, resample=False, expand=False, center=None),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor()
                                ])

transform_test  = transforms.Compose([transforms.Resize(224),
                                 transforms.ToTensor()
                                ])

train_dataset = MyDataset(csv_file=labels, data=train_idx, transform=transform_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

valid_dataset = MyDataset(csv_file=labels, data=test_idx, transform=transform_test)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=4)

test_dataset = MyDataset(csv_file=labels, data=test_data, transform=transform_test, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import fastprogress
from fastprogress import master_bar, progress_bar

class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func=func

    def forward(self, x): return self.func(x)
    
def Flatten():
    return Lambda(lambda x: x.view((x.size(0), -1)))

def train(mb, model, device, train_loader, optimizer, epoch, loss_fn, scheduler=None):
    model.train()
    train_loss = 0
    if scheduler:
      scheduler.step()
    for batch_idx, (data, target) in enumerate(progress_bar(train_loader, parent=mb)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        if scheduler:
          scheduler.batch_step()
        mb.child.comment = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item())
    
    return train_loss/len(train_loader)
        
def test(mb, model, device, test_loader, loss_fn):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    return correct/len(test_loader.dataset), test_loss
    
def evaluate(model, device, test_loader):
    model.eval()
    preds = []
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            output = model(data)
            pred = output.max(1, keepdim=True)[1].tolist() # get the index of the max log-probability
            preds += pred
    return preds

In [ ]:
from torch.autograd import Variable

### Focal loss
def one_hot(index, classes):
    size = index.size() + (classes,)
    view = index.size() + (1,)

    mask = torch.Tensor(*size).fill_(0).cuda()
    index = index.view(*view)
    ones = 1.

    if isinstance(index, Variable):
        ones = Variable(torch.Tensor(index.size()).fill_(1)).cuda()
        mask = Variable(mask, volatile=index.volatile).cuda()

    return mask.scatter_(1, index, ones)


class FocalLoss(nn.Module):

    def __init__(self, gamma=2, alpha=0.25, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
        self.alpha = alpha

    def forward(self, input, target, reduction=None):
        y = one_hot(target, input.size(-1))
        logit = F.softmax(input, dim=-1)
        logit = logit.clamp(self.eps, 1. - self.eps)

        loss = -1 * y * torch.log(logit) # cross entropy
        loss = self.alpha * loss * (1 - logit) ** self.gamma # focal loss
        if reduction:
          return loss.sum()
        else:
          return loss.mean()

In [ ]:
### hyper-parameters
lr=1e-2
momentum=0.9
epochs = 40

### loss functions
# loss_fn = F.cross_entropy
loss_fn = FocalLoss()

In [ ]:
import copy
from torchvision.models import resnet18, resnet50, densenet121, densenet169
import pdb
from adamw import AdamW
from cosine_scheduler import CosineLRWithRestarts

device = torch.device("cuda")

###resnet 50
# model = resnet50(pretrained=False)

###resnet 18
# model = resnet18(pretrained=False)

###resnet layers
# model.avgpool = nn.AdaptiveAvgPool2d(1)
# model.fc = nn.Linear(model.fc.in_features, 31)


# model = nn.Sequential(*module_list)
# features = model[:-4]
# fc = model[-4:]

### densenet 121
model = densenet121(pretrained=False)
model.classifier = nn.Linear(model.classifier.in_features, 31)

###sequential model 2 hidden layer just for test
# model = nn.Sequential(
#             Flatten(),
#             nn.Linear(10000, 3000),
#             nn.Sigmoid(),
#             nn.Linear(3000, 512),
#             nn.Sigmoid(),
#             nn.Linear(512,31))
# model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 31)
model = model.to(device)


### SGD with momentum with regularization
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=5e-4, nesterov=True)

### SGD 
# optimizer = optim.SGD(model.parameters(), lr=lr)

### Adam optimizer with 2 diffeerent learing rate at different layers
# optimizer =AdamW([
#                 {'params': model.features.parameters(), 'lr': lr/10},
#                 {'params': model.classifier.parameters(), 'lr': lr/100}
#                 ],
#                 weight_decay=5e-4)


### Adam optimizer with constant learning rate
optimizer = optim.Adam(model.parameters(), lr=lr/10)

### Adam optimzer constant leanring rate and reqularization
# optimizer = AdamW(model.parameters(), lr=lr/10, weight_decay=5e-4)


### scheduler on/off
# scheduler=None
scheduler = CosineLRWithRestarts(optimizer, 32, len(train_loader.dataset), restart_period=5, t_mult=1.2)


### main code 
mb = master_bar(range(1, epochs + 1))
best_acc = 0
best_model_dict = {}

trn_losses = []
test_losses = []
accs = []

for epoch in mb:
    train_loss = train(mb, model, device, train_loader, optimizer, epoch, loss_fn, scheduler=scheduler)
    acc, test_loss = test(mb, model, device, valid_loader, loss_fn)
    mb.write(f'Epoch {epoch}, Train loss {train_loss:.4f}, Test loss: {test_loss:.4f}, Accuracy: {100*acc:.0f}%')
    
    trn_losses.append(train_loss)
    test_losses.append(test_loss)
    accs.append(acc*100)


    if acc > best_acc:
        best_acc=acc
        best_model_dict = copy.deepcopy(model.state_dict())

In [ ]:
import matplotlib.pyplot as plt
plt.subplot(211)
plt.plot(trn_losses, label='train_loss')
plt.plot(test_losses, label='test_loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend()
plt.subplot(212)
plt.plot(accs, label='accuracy')
plt.xlabel('epochs')
plt.subplots_adjust(hspace=0.5)

plt.savefig('trn_test_losses.png')

In [ ]:
model.load_state_dict(best_model_dict)
preds = evaluate(model, device, test_loader)

In [ ]:
lbs = pd.read_csv(labels)
_labels = lbs['Category'].values.tolist()
_classes = sorted(set(lbs['Category'].values.tolist()))
_idx2class = {idx:cl for idx, cl in enumerate(_classes)}

In [ ]:
idxs = []
for idx in np.asarray(preds).reshape(-1):
    idxs.append(_idx2class[idx])

In [ ]:
s = pd.Series(idxs, index=range(len(preds)))

In [ ]:
df = s.to_frame().reset_index()
df.columns = ['Id', 'Category']

In [ ]:
df.to_csv('out.csv', index = False)